In [1]:
import pygame
FPS = 30
class Ant:
    def __init__(self, x, y, direction, ant_id):
        self.x = x
        self.y = y
        self.direction = direction
        self.id = ant_id

    def turn_clockwise(self):
        self.direction = (self.direction + 1) % 4

    def turn_counterclockwise(self):
        self.direction = (self.direction - 1) % 4

    def move_forward(self):
        if self.direction == 0:    # Up
            self.y -= 1
        elif self.direction == 1:  # Right
            self.x += 1
        elif self.direction == 2:  # Down
            self.y += 1
        else:                      # Left
            self.x -= 1

class LangtonAnts:
    def __init__(self, width, height, cell_size):
        pygame.init()
        
        # Initialize display
        self.width = width
        self.height = height
        self.cell_size = cell_size
        # Add extra height for the counter
        self.screen = pygame.display.set_mode((width * cell_size, height * cell_size + 30))
        pygame.display.set_caption("Langton's Ants")
        
        self.font = pygame.font.Font(None, 36)
        self.iterations = 0
        
        # Initialize grids
        self.grid = [[0 for _ in range(width)] for _ in range(height)]
        self.pheromone_grid = [[None for _ in range(width)] for _ in range(height)]
        
        # Create ants
        self.ants = [
            Ant(width // 3, height // 2, 0, 'A'),
            Ant(2 * width // 3, height // 2, 0, 'B')
        ]
        
        # Initialize clock
        self.clock = pygame.time.Clock()
        
        # Pheromone properties
        self.pheromone_strength = {}  # Store pheromone strength
        self.PHEROMONE_DECAY = 0.05
        self.SELF_PHEROMONE_PROB = 0.8

    def update(self):
        # Update pheromones
        for y in range(self.height):
            for x in range(self.width):
                if (x, y) in self.pheromone_strength:
                    self.pheromone_strength[(x, y)] -= self.PHEROMONE_DECAY
                    if self.pheromone_strength[(x, y)] <= 0:
                        del self.pheromone_strength[(x, y)]
                        self.pheromone_grid[y][x] = None

        # Update ants
        for ant in self.ants:
            # Wrap around edges
            ant.x = ant.x % self.width
            ant.y = ant.y % self.height
            
            # Check current cell
            current_cell = self.grid[ant.y][ant.x]
            current_pheromone = self.pheromone_grid[ant.y][ant.x]
            
            # Determine movement based on pheromones
            follow_standard_rules = True
            if current_pheromone:
                if (ant.x, ant.y) in self.pheromone_strength:
                    strength = self.pheromone_strength[(ant.x, ant.y)]
                    if current_pheromone == ant.id:  # Own pheromone
                        if pygame.time.get_ticks() % 100 < self.SELF_PHEROMONE_PROB * 100 * strength:
                            follow_standard_rules = False
                    else:  # Other pheromone
                        if pygame.time.get_ticks() % 100 >= self.SELF_PHEROMONE_PROB * 100 * strength:
                            follow_standard_rules = False

            # Apply movement rules
            if follow_standard_rules:
                if current_cell == 0:  # White
                    ant.turn_clockwise()
                else:  # Black
                    ant.turn_counterclockwise()
            
            # Flip cell color
            self.grid[ant.y][ant.x] = 1 - current_cell
            
            # Leave pheromone
            self.pheromone_grid[ant.y][ant.x] = ant.id
            self.pheromone_strength[(ant.x, ant.y)] = 1.0
            
            # Move forward
            ant.move_forward()
        
        # Increment iteration counter
        self.iterations += 1

    def draw(self):
        self.screen.fill((255, 255, 255))  # White background
        
        # Draw grid and pheromones
        for y in range(self.height):
            for x in range(self.width):
                # Draw cell
                if self.grid[y][x] == 1:
                    pygame.draw.rect(self.screen, (0, 0, 0),
                                  (x * self.cell_size, y * self.cell_size,
                                   self.cell_size, self.cell_size))
                
                # Draw pheromone
                if (x, y) in self.pheromone_strength:
                    strength = self.pheromone_strength[(x, y)]
                    if strength > 0:
                        color = (255, 0, 0) if self.pheromone_grid[y][x] == 'A' else (0, 0, 255)
                        surface = pygame.Surface((self.cell_size, self.cell_size))
                        surface.set_alpha(int(128 * strength))
                        surface.fill(color)
                        self.screen.blit(surface, (x * self.cell_size, y * self.cell_size))
        
        # Draw ants
        for ant in self.ants:
            color = (255, 0, 0) if ant.id == 'A' else (0, 0, 255)
            pygame.draw.circle(self.screen, color,
                            (ant.x * self.cell_size + self.cell_size // 2,
                             ant.y * self.cell_size + self.cell_size // 2),
                            self.cell_size // 3)
        
        # Iteration counter
        counter_text = f"Iterations: {self.iterations}"
        text_surface = self.font.render(counter_text, True, (0, 0, 0))
        text_rect = text_surface.get_rect(center=(self.width * self.cell_size // 2, self.height * self.cell_size + 15))
        self.screen.blit(text_surface, text_rect)
        
        pygame.display.flip()

    def run(self):
        running = True
        while running:
            # Handle events
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        running = False
            
            # Update and draw
            self.update()
            self.draw()
            self.clock.tick(FPS)
        
        pygame.quit()



simulation = LangtonAnts(width=100, height=80, cell_size=8)
simulation.run()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
